In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import math
import sympy as sp

In [29]:
# Perform inverse kinematics for an in-plane mammal geometry leg.
# This will be used as a component of the full 3dof IK solution.

# This is conducted purely in 2d.  The geometry looks like:
#
#     O               +y
#    /                ^
#   /  <--- femur     |
#  /
#  ---------P        -> +x
#
#    ^tibia
#
# Where O is the origin and P is the end effector point in the 
# y/z coordinate frame.  +y is to the right in this diagram and
# +z is up.

# Px, Py - position of end effector
# l1, l2 - length of femur and tibia
# theta1 - angle from vertical of upper leg
# theta2 - angle from vertical of lower leg
Px, Py, l1, l2, theta1, theta2 = sp.symbols('Px Py l1 l2 theta1 theta2')

# This is a simple triangle.  We know the length of all three sides,
# which means we can find all three angles.

# Use the law of cosines to find the tibia angle first.  Note: A
# practical implementation of this needs to handle the limited domain
# of the relevant actuators and dimensions.
norm_PO_sq = Px * Px + Py * Py
norm_PO = sp.sqrt(norm_PO_sq)

cos_tibia_sub = l1 * l1 + l2 * l2 - norm_PO_sq

cos_tibia_inv = cos_tibia_sub / (2 * l1 * l2)
tibia_angle = sp.acos(cos_tibia_inv)

# Now do the femur.
cos_femur_sub = norm_PO_sq + l1 * l1 - l2 * l2
cos_femur_inv = cos_femur_sub / (2 * norm_PO * l1)
femur_angle = sp.acos(cos_femur_inv)

# Finally, convert these triangle angles into theta1 and theta2,
# which are both measured relative to vertical.
theta1 = sp.atan2(Py, Px) + femur_angle + sp.pi / 2
theta2 = theta1 - tibia_angle
theta1, theta2


(acos((Px**2 + Py**2 + l1**2 - l2**2)/(2*l1*sqrt(Px**2 + Py**2))) + atan2(Py, Px) + pi/2,
 -acos((-Px**2 - Py**2 + l1**2 + l2**2)/(2*l1*l2)) + acos((Px**2 + Py**2 + l1**2 - l2**2)/(2*l1*sqrt(Px**2 + Py**2))) + atan2(Py, Px) + pi/2)

In [5]:
# Calculate the torque required from an in-plane mammal geometry leg
# to achieve a given end-effector force.  We assume that the 3rd degree
# of freedom is not a significant contributor to torque requirements.

# Fx, Fy - force at end effector
# t1, t2 - torque on the upper and lower leg
# l1, l2 - length of upper and lower leg
# theta1 - angle from vertical of upper leg
# theta2 - angle from vertical of lower leg
#
# Reference free body diagram at: 
# https://docs.google.com/drawings/d/19iMuQCkOq_pnOxSgBEZ9HpjowesbO7r2yoPeQHNCyjA/edit?usp=sharing
Fx, Fy, t1, t2, l1, l2, theta1, theta2 = \
   sp.symbols('Fx Fy t1 t2 l1 l2 theta1 theta2')
    
# We can trivially determine that F = I, so we'll just use F for I everywhere.
    
# First, generate the equations for the top member.
    
# Torque as measured from the top pin point.
top_torque = sp.Eq(t1 + t2 - sp.cos(theta1) * l1 * Fx - sp.sin(theta1) * l1 * Fy, 0)

# The bottom force equation is identical to the top, so provides no more 
# degrees of freedom.

# Torque as measured from the bottom pin point.
bottom_torque = sp.Eq(t2 - sp.cos(theta2) * l2 * Fx + sp.sin(theta2) * l2 * Fy, 0)

# Now we have 2 equations and two unknowns.
torque_equations = sp.solve([top_torque, bottom_torque], [t1, t2])
torque_equations

{t1: Fx*l1*cos(theta1) - Fx*l2*cos(theta2) + Fy*l1*sin(theta1) + Fy*l2*sin(theta2),
 t2: l2*(Fx*cos(theta2) - Fy*sin(theta2))}

In [30]:
sl1 = 0.113
sl2 = 0.119
sPx = 0
sPy = -0.20
theta2.subs([(l1, sl1), (l2, sl2), (Px, sPx), (Py, sPy)])


#r2 = {key: value.subs(theta1, math.radians(30)).subs(theta2, math.radians(15)).subs(l1, sl1).subs(l2, sl2).subs(Fx, 0)
#  for key, value in r.items()}
#r2


-1.53141840691676